In [6]:
import re
from pymongo import MongoClient 

#預設就是自己
client = MongoClient()
database = client['test']
collection =database['ptt']

import jieba
jieba.set_dictionary('D:/TextMining/dict.txt.big.txt')  #切換至中文繁體字庫
jieba.load_userdict("D:/TextMining/dict_keyw.txt")       #加入自建詞庫
jieba.load_userdict("D:/TextMining/ptt.txt")       #加入自建詞庫

Building prefix dict from D:\TextMining\dict.txt.big.txt ...
Loading model from cache c:\users\ytchen\appdata\local\temp\jieba.ufb55ea6623e696143f43e248198acd74.cache
Loading model cost 1.375 seconds.
Prefix dict has been built succesfully.


In [48]:
#給關鍵字----------------------------------------
tag1 = "烤肉"
# tag2 = "台灣"
#-----------------------------------------------

title=[]
push_content =[]
jieba_content = []
all_article = []
all_id_pushContent = []
#把資料庫東西抓出來
for post in collection.find(
    {"$or":[                   
            {"main_content":{"$regex":tag1}},
            {"title":{"$regex":tag1}},
#             {"push_content.content":{"$regex":tag1}},
            ],
     "category":"新聞"},{"_id":0}).limit(20):
    
    #store news data
    id_pushContent = [] #一篇文章中的所有推文
    title.append(post['title'])
    summary = ''
    for i in post['push_content']:
        summary += '#'+i['content'] #以井號連接每篇推文 
        #記綠推文id & 推文
        temp_data = {
            'userid':i['userid'],
            'push':i['content'],
        }
        id_pushContent.append(temp_data)
    all_id_pushContent.append(id_pushContent) #將每篇文章的推文資訊存取
    all_article.append(summary)
    jieba_content.append(' '.join(jieba.cut(summary)))
    
    print post['title'],post['url']
#     for i in post['push_content']:
#         print i['userid'],i['content']


#文章數
newsNumber = len(title)
print "文章數:" + str(newsNumber)

[新聞] 偷摸女子下體被眾人痛毆 告殺人未遂檢察 https://www.ptt.cc/bbs/Gossiping/M.1472474604.A.0E7.html
[新聞] CNN評選全球最佳烤肉　台灣蒙古烤肉上榜 https://www.ptt.cc/bbs/Gossiping/M.1472568789.A.E3A.html
Re: [新聞] CNN評選全球最佳烤肉　台灣蒙古烤肉上榜 https://www.ptt.cc/bbs/Gossiping/M.1472571031.A.781.html
Re: [新聞] CNN評選全球最佳烤肉　台灣蒙古烤肉上榜 https://www.ptt.cc/bbs/Gossiping/M.1472572232.A.446.html
[新聞] 「蒙古烤肉」來自台灣 入選CNN最佳烤肉긱 https://www.ptt.cc/bbs/Gossiping/M.1472573995.A.BEA.html
Re: [新聞] 「蒙古烤肉」來自台灣 入選CNN最佳烤肉1 https://www.ptt.cc/bbs/Gossiping/M.1472575359.A.8CE.html
Re: [新聞] 「蒙古烤肉」來自台灣 入選CNN最佳烤肉1 https://www.ptt.cc/bbs/Gossiping/M.1472577672.A.49F.html
Re: [新聞] 「蒙古烤肉」來自台灣 入選CNN最佳烤肉1 https://www.ptt.cc/bbs/Gossiping/M.1472578748.A.849.html
Re: [新聞] 「蒙古烤肉」來自台灣 入選CNN最佳烤肉1 https://www.ptt.cc/bbs/Gossiping/M.1472580473.A.32D.html
Re: [新聞] 「蒙古烤肉」來自台灣 入選CNN最佳烤肉1 https://www.ptt.cc/bbs/Gossiping/M.1472581440.A.A2A.html
Re: [新聞] 「蒙古烤肉」來自台灣 入選CNN最佳烤肉1 https://www.ptt.cc/bbs/Gossiping/M.1472582505.A.5E6.html
[新聞] 女孩兒喜歡「X老二」嗎？　她吃完大驚 https://www.ptt.cc/bbs/Go

## TF-IDF權重計算

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(jieba_content)  # titile 放文本
weight = X.toarray() #裝關鍵字的權重值

#特徵值總共有多少個
features = vectorizer.get_feature_names()     # 拿到所有的關鍵詞  
print "特徵值數量:",len(features)


特徵值數量: 1305


## 拿到每篇文章的TOP TF-IDF

In [50]:
import time
tic = time.clock()

top_features = []
final_top_features = []
for n in range(0,newsNumber):  #迴圈參考上面的總文章數
    #裝每天文章的feature & score
    map_top_features = []
    
    #對特徵詞的index進行排序
    indices = np.argsort(weight[n])[::-1] #weight[n] 是第幾篇新聞的意思
    
    #拿到所有的特徵詞
    features = vectorizer.get_feature_names()
    
    #截取TOP多少的詞
    top_n = 40
    
    ###################################把每篇的TOP TF-IDF 與其分數存起來###############################################
    #將每天文章的TOP-TFIDF存起來
    top_features.append([features[i] for i in indices[:top_n]])
    
    a=0
    for i in top_features[n]:
#         print i,weight[n][indices[a]]
        data = {
            'feature':i,
            'score':weight[n][indices[a]]
        }
        #print data['score'],data['feature']
        a=a+1
        
        map_top_features.append(data)
    final_top_features.append(map_top_features)
# print final_top_features[0]
    #############################################################################################
    
toc = time.clock()
print "執行時間:",(toc - tic),"秒"

執行時間: 0.0344665857376 秒


## PTT版文章摘要運算

In [57]:
##新版文章摘要
for article_index in range(0,newsNumber):
   
    #容器設定
    id_sen_score_map = []  #裝一篇文章的句子 & 句子的分數
    article_score = 0 #文章的總分數
    avg_score = 0 #總文章的平均分數
    
    #推文分數計算
    article = all_article[article_index] #一篇文章
    temp_id_pushContent = all_id_pushContent[article_index] #一篇文章的推文資訊
    article_sen_size = len(temp_id_pushContent) #推文數量
    
    for j in temp_id_pushContent:
        sen =  j['push'].encode('utf-8')
        userid = j['userid'].encode('utf-8')
        
        #初始化句子分數
        score = 0 
        #逐一抓出各文章的 map_top_features 特徵詞與分數，與句子進行比對，若該特徵詞存在於句子中，則分數加上去
        for i in final_top_features[article_index]:
            feature = i['feature'].encode('utf-8')
            if feature in sen:
                score += i['score']
        
        #存取句子與分數
        data = {
            'userid':userid,
            'sentence' : sen,
            'score' : score,
        }
        id_sen_score_map.append(data)
        
        #文章總分數
        article_score += score        
        
    #文章資訊
    avg_score = article_score / article_sen_size #平均句子分數
    

    #文章摘要
    articleSummarize = ''
    skip = 0
    for i in id_sen_score_map:
        #進行判斷(句子分數 > 平均分數才加入摘要)
        if i['score'] > avg_score*1.2:
            articleSummarize += i['userid']+":"+i['sentence']+' '
    #修整文章摘要
    articleSummarize.strip()
            
    print '平均分數=' + str(avg_score)
    print '---------------------------------------------'
    print '新聞標題：'+title[article_index].encode('utf-8')
    print '------------'
    print '文章摘要：'
    print articleSummarize
    print '------------'
    print '推文數量:'+ str(article_sen_size)
    print '文章原文：'
    print article
    print '---------------------------------------------'    


平均分數=0.31737111608
---------------------------------------------
新聞標題：[新聞] 偷摸女子下體被眾人痛毆 告殺人未遂檢察
------------
文章摘要：
Skydier:八卦人氣 -1 HC683150:五樓幹的，真噁 liudwan:新北上色推回來 jhjhs33504:五樓... nebbiabards:五樓自首了 zxlt3722:新北沒上色 makimakimaki:這告得成 以後看到有人性侵只能柔性勸告了 canblow:五樓又你 s902131:看到有人性侵是不會叫警察喔 su4vu6:應該告傷害吧 怎會告殺人未遂 HwaSIn:希望法官女兒被性侵 旁人都袖手旁觀 JamesloveTai:怒噓五樓 kingyes:實在不知道某人在說甚麼 被起訴跟法官有甚麼關係 sazabik:五樓 免費的最貴～ 
------------
推文數量:35
文章原文：
#新北#八卦人氣 -1#台女不意外#人氣-1#五樓幹的，真噁#打得好#八卦-1#又是5樓 唉#新北上色推回來#又是亂吉#原來有義憤傷害罪#-1#五樓...#八卦板-1#新北   ㄏㄏ#八卦-1#五樓自首了#黑青瘀傷也要告殺人未遂喔#台灣治安怎麼那麼糞?#新北沒上色#新北日常#這告得成 以後看到有人性侵只能柔性勸告了#叫泥帶本票偏偏不聽話#五樓又你#摸一下屁股判6個月，有事嗎#看到有人性侵是不會叫警察喔#一定很正#應該告傷害吧 怎會告殺人未遂#告傷害應該能成立#其他人憑什麼動用私刑？#希望法官女兒被性侵 旁人都袖手旁觀#怒噓五樓#實在不知道某人在說甚麼 被起訴跟法官有甚麼關係#五樓 免費的最貴～#現行犯不是可以逮捕
---------------------------------------------
平均分數=0.542134139905
---------------------------------------------
新聞標題：[新聞] CNN評選全球最佳烤肉　台灣蒙古烤肉上榜
------------
文章摘要：
elqqaapple:蒙古是台灣的！OuO zhttp:中華台北 vs 台灣蒙古 kevin0515:原來蒙古烤肉是台灣的... ams9:蒙古是台灣不可分割的一部分 linfon00:蒙古烤肉.....

## 新聞版文章摘要

In [44]:
#對所有的新聞文章進行 "句子切詞 "
import re
for article_index in range(0,newsNumber):
    article = all_article[article_index] #一篇文章
    setenceList = re.split("#",article.encode('utf-8')) #根據"逗號" | "句號" | \n 進行句子斷開
    
    #裝一篇文章的句子 & 句子的分數
    sen_score_map = []
    article_score = 0 #文章的總分數
    avg_score = 0 #總文章的平均分數
    
    #將每篇文章的句子逐一取出進行分數計算
    for sen in setenceList:
        #初始化句子分數
        score = 0 
        
        #逐一抓出各文章的 map_top_features 特徵詞與分數，與句子進行比對，若該特徵詞存在於句子中，則分數加上去
        for i in final_top_features[article_index]:
            feature = i['feature'].encode('utf-8')
            if feature in sen:
                score += i['score']
        
        #存取句子與分數
        data = {
            'sentence' : sen,
            'score' : score,
        }
        sen_score_map.append(data)
        
        #文章總分數
        article_score += score
        
    #文章資訊
    article_sen_size = len(setenceList) #文章句子總數
    avg_score = article_score / article_sen_size #平均句子分數
    
    #文章摘要
    articleSummarize = ''
    skip = 0
    for i in sen_score_map:
        #進行判斷(句子分數 > 平均分數才加入摘要)
        if i['score'] > avg_score*1.2:
            articleSummarize += i['sentence']+' '
    #修整文章摘要
    articleSummarize.strip()
            
    print '平均分數=' + str(avg_score)
    print '---------------------------------------------'
    print '新聞標題：'+title[article_index].encode('utf-8')
    print '------------'
    print '文章摘要：'
    print articleSummarize
    print '------------'
    print '文章原文：'
    print article
    print '---------------------------------------------'    

平均分數=0.595020782913
---------------------------------------------
新聞標題：[新聞] 中共伸黑手 電影《十年》映港人憂慮
------------
文章摘要：
這部片很好看，但是內容很沈重。 來台灣上映吧！香港已經變成中國的形狀了。 台灣已經上映了，但廳數很少。 已經上映了好嗎？ ="= 這部片不是腦補 現在中共正在思想清洗香港人 
------------
文章原文：
#十年之前 你不記得我#這部片很好看，但是內容很沈重。#腦補劇情#十年之後 我們是碰友#來台灣上映吧！香港已經變成中國的形狀了。#台灣已經上映了，但廳數很少。#廳數少，不過很多場都滿就是了#好看推，快去電影院看#殺支那共匪賤畜十億zz#已經上映了好嗎？ ="=#這部片不是腦補 現在中共正在思想清洗香港人#種子都滿天飛了#可以看dailymotion
---------------------------------------------
平均分數=0.404678612475
---------------------------------------------
新聞標題：Re: [新聞] 陸客不來糟了？7月觀光客總數多1萬人
------------
文章摘要：
入境人口可能是來出差的 不一定能跟觀光畫等號 所以近60萬的基數在刻意操作之下近程同期相比少五千人 就蛆急嗨說觀光要死掉是死幾點的？ 你能從簽證判斷自由行的跟出差的區別？我知道的是來開 會除了正式的大頭以外常常報自由行免往上送審 來台也不一定持光觀簽證啊 主要還是以觀光為主吧 
------------
文章原文：
#NCC:沒空，我忙著在抓動畫人物抽煙#喔耶 中華民國又要滅亡了#1%的差距有什麼好講故事的啊 我每天起床時間誤差都超過1%#原文是2% 也是大驚小怪#捲舌音不來最棒#作假被抓到了 大家快裝死#看你切哪一天分界的同期#入境人口可能是來出差的 不一定能跟觀光畫等號#所以近60萬的基數在刻意操作之下近程同期相比少五千人#就蛆急嗨說觀光要死掉是死幾點的？#你能從簽證判斷自由行的跟出差的區別？我知道的是來開#會除了正式的大頭以外常常報自由行免往上送審#一堆看報聽說發評論的，以為普通導遊說的是聖旨#來台旅客人數的確多了一萬多，可上觀光局查資料#來台也不一定持光觀簽證

In [28]:
import re
s = '今天天氣 真的超級好，很喜歡。不過阿我還是\n比較喜歡'
k = re.split("，|。|\n",s) 
for i in k :
    print i

今天天氣 真的超級好
很喜歡
不過阿我還是
比較喜歡


In [1]:
test = ''
s = ['a','b','c','d']
test = '#'.join(s)
print test
print type(test)

a#b#c#d
<type 'str'>


In [59]:
article = {
    'comp':'Gossip',
    'category':'問卦',
    'url':'',
    'author':'',
    'title':'',
    'date':'',
    'push_nums':'',
    'main_content':'',
    'push_content':[  #推文會有多個，所以用list裝
        {
            'tag':'',
            'userid':'',
            'datetime':'',
            'content':'456',
        }
    ],
    'push_count':{  #算數量的只會有一個，用字典裝就好
            'all':'',
            'good':'',
            'bad':'',
            'neutral':'',
            'my_push_nums':'good-bad',
    }
    
}
print article

print article['push_content']
print article['push_content'][0]['content']

{'category': '\xe5\x95\x8f\xe5\x8d\xa6', 'date': '', 'push_count': {'my_push_nums': 'good-bad', 'all': '', 'good': '', 'neutral': '', 'bad': ''}, 'push_content': [{'content': '456', 'tag': '', 'userid': '', 'datetime': ''}], 'push_nums': '', 'author': '', 'url': '', 'comp': 'Gossip', 'title': '', 'main_content': ''}
[{'content': '456', 'tag': '', 'userid': '', 'datetime': ''}]
456


In [9]:
a = [1,2,3,4,8,98]
b = [8,5,4,5,1,2,5]
c = [a,b]

In [10]:
print c

[[1, 2, 3, 4, 8, 98], [8, 5, 4, 5, 1, 2, 5]]
